In [1]:
! pip install pandas
! pip install petl
! pip install glob


You should consider upgrading via the 'C:\Users\adamc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\adamc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.
ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob
You should consider upgrading via the 'C:\Users\adamc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import petl as etl
import os
import glob

1. Load CSV files to dataframes

In [3]:
csv_files_data = pd.read_csv('data_output/1_csv_files_dataframe.csv')
json_files_data = pd.read_csv('data_output/1_json_files_dataframe.csv')
sales_df = csv_files_data.append(json_files_data)


2. Prepare data for visual 1

In [4]:
# a) group data

sales_df_v1 = sales_df
sales_df_v1 = sales_df_v1.groupby(['branch', 'product']).sum().reset_index()

,branch,product,year,month,day,hour,amount_in_gbp,qty
0,Armagh outlet,21 day mature ribeye steak,14680393,47737,107311,82749,2727963.140,847509
1,Armagh outlet,21 day mature rump steak,14537196,47277,104585,83473,2478042.176,842220
2,Armagh outlet,21 day mature sirlion steak,14638462,48033,105978,83492,2641329.780,844288
3,Armagh outlet,Almond milk,14535049,46865,104482,82188,1486600.052,851103
4,Armagh outlet,Blueberry yoghurt,14882113,48710,106852,85092,875294.684,864310


In [5]:
# b) convert to etl table and cut out not needed fields
sales_df_v1 = etl.fromdataframe(sales_df_v1)
sales_df_v1 = etl.transform.cutout(sales_df_v1, 'year', 'month', 'day', 'hour')

In [6]:
# c) lookup region and county

# branch list load
branch_list = etl.fromxlsx('branch_data/branch_list.xlsx')
branch_list = branch_list.rename({'branch_name': 'branch'})

# branch list join
sales_df_v1 = etl.outerjoin(sales_df_v1, branch_list)
sales_df_v1 = etl.transform.cutout(sales_df_v1, 'established_on')

In [7]:
# d) lookup product category

# product list load
product_list = etl.fromcsv('branch_data/products_list.csv')
product_list=etl.transform.cut(product_list, 'product', 'category')

# product list join
sales_df_v1 = etl.rightjoin(sales_df_v1, product_list, 'product')

In [8]:
# e) save to csv
etl.tocsv(sales_df_v1, 'data_output/2_visuals_pt1.csv')

3. Prepare data for visual 2

In [19]:
# a) group data

sales_df_v2 = sales_df
sales_df_v2 = sales_df_v2.groupby(['branch']).sum().reset_index()

,branch,year,month,day,hour,amount_in_gbp,qty
0,Armagh outlet,1815562950,5913261,13164158,10324665,1.135290e+10,105239104
1,Ballymoney store,1828570261,5910767,12988533,10447031,1.145031e+10,106169826
2,Bargoed outlet,1830846791,5881006,13414310,10433361,1.148845e+10,106212042
3,Barry outlet,2048262992,6568293,14917520,11665778,1.269032e+10,118919062
4,Bassetlaw outlet,2256117614,7236424,16237355,12880923,1.383844e+10,131010110


In [20]:
# b) convert to etl table and cutout not needed columns
sales_df_v2 = etl.fromdataframe(sales_df_v2)
sales_df_v2 = etl.transform.cutout(sales_df_v2, 'year', 'month', 'day', 'hour')

branch,amount_in_gbp,qty
Armagh outlet,11352899748.7514,105239104
Ballymoney store,11450307515.861,106169826
Bargoed outlet,11488453160.198,106212042
Barry outlet,12690322614.757,118919062
Bassetlaw outlet,13838435781.5576,131010110


In [21]:
# c) lookup region and county

# branch list load
branch_list = etl.fromxlsx('branch_data/branch_list.xlsx')

# branch list join
sales_df_v2 = etl.outerjoin(sales_df_v2, branch_list, lkey='branch', rkey='branch_name')
sales_df_v2 = etl.transform.cutout(sales_df_v2, 'established_on')

+------------------+------------------+-----------+------------------+------------+
| branch           | amount_in_gbp    | qty       | region           | county     |
+==================+==================+===========+==================+============+
| Armagh outlet    | 11352899748.7514 | 105239104 | Northern Ireland | Armagh     |
+------------------+------------------+-----------+------------------+------------+
| Ballymoney store |  11450307515.861 | 106169826 | Northern Ireland | Ballymoney |
+------------------+------------------+-----------+------------------+------------+
| Bargoed outlet   |  11488453160.198 | 106212042 | Wales            | Bargoed    |
+------------------+------------------+-----------+------------------+------------+
| Barry outlet     |  12690322614.757 | 118919062 | Wales            | Barry      |
+------------------+------------------+-----------+------------------+------------+
| Bassetlaw outlet | 13838435781.5576 | 131010110 | East Midlands    | Basse

In [22]:
# d) save to csv
etl.tocsv(sales_df_v2, 'data_output/2_visuals_pt2.csv')

4. Prepare data for visual 3

In [34]:
# a) group and sum data
sales_df_v3 = sales_df
sales_df_v3 = sales_df_v3.groupby(['branch', 'year', 'month', 'day', 'hour']).sum().reset_index()

,branch,year,month,day,hour,amount_in_gbp,qty
0,Armagh outlet,2012,1,2,16,6.807334e+06,80196
1,Armagh outlet,2012,1,7,18,1.603301e+06,7032
2,Armagh outlet,2012,1,9,13,3.229247e+06,34377
3,Armagh outlet,2012,1,10,0,5.724104e+06,61828
4,Armagh outlet,2012,1,11,8,3.626819e+06,47488


In [35]:
# b) group by branch and get average
sales_df_v3 = sales_df_v3.groupby(['branch']).mean().reset_index()

,branch,year,month,day,hour,amount_in_gbp,qty
0,Armagh outlet,2016.0,6.5,14.522762,11.5,4.379977e+06,40601.506173
1,Ballymoney store,2016.0,6.5,14.283179,11.5,4.417557e+06,40960.581019
2,Bargoed outlet,2016.0,6.5,14.727623,11.5,4.432274e+06,40976.868056
3,Barry outlet,2015.5,6.5,14.556597,11.5,4.406362e+06,41291.340972
4,Bassetlaw outlet,2015.0,6.5,14.530934,11.5,4.368193e+06,41354.201389


In [36]:
# c) coutout (drop) columns 
sales_df_v3 = sales_df_v3.drop(['year', 'month', 'day', 'hour', 'qty'], axis=1)

,branch,amount_in_gbp
0,Armagh outlet,4.379977e+06
1,Ballymoney store,4.417557e+06
2,Bargoed outlet,4.432274e+06
3,Barry outlet,4.406362e+06
4,Bassetlaw outlet,4.368193e+06


In [37]:
# d) save to csv
sales_df_v3.to_csv('data_output/2_visuals_pt3.csv')